In [1]:
import os
import numpy as np
import cv2
import tensorflow as tf
from glob import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard


# Create Model using TensorFlow Functional API

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Input, MaxPool2D, UpSampling2D, Concatenate
from tensorflow.keras.models import Model


def conv_block(x, num_filters):
    x = Conv2D(num_filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x


def build_model():
    size = 256
    num_filters = [16, 32, 48, 64]
    inputs = Input((size, size, 3))

    skip_x = []
    x = inputs
    # Encoder
    for f in num_filters:
        x = conv_block(x, f)
        skip_x.append(x)
        x = MaxPool2D((2, 2))(x)

    # Bridge
    x = conv_block(x, num_filters[-1])

    num_filters.reverse()
    skip_x.reverse()
    # Decoder
    for i, f in enumerate(num_filters):
        x = UpSampling2D((2, 2))(x)
        xs = skip_x[i]
        x = Concatenate()([x, xs])
        x = conv_block(x, f)

    # Output
    x = Conv2D(1, (1, 1), padding="same")(x)
    x = Activation("sigmoid")(x)

    return Model(inputs, x)


model = build_model()
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 256, 16) 64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 256, 256, 16) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

# Load Train Filenames

In [3]:
def load_data(path, split=0.1):
    images = sorted(glob(os.path.join(path, "images/*")))
    masks = sorted(glob(os.path.join(path, "masks/*")))

    total_size = len(images)
    valid_size = int(split * total_size)
    test_size = int(split * total_size)

    train_x, valid_x = train_test_split(
        images, test_size=valid_size, random_state=42)
    train_y, valid_y = train_test_split(
        masks, test_size=valid_size, random_state=42)

    train_x, test_x = train_test_split(
        train_x, test_size=test_size, random_state=42)
    train_y, test_y = train_test_split(
        train_y, test_size=test_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)


# Loading Train Data

In [4]:
import os
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
from sklearn.model_selection import train_test_split


## Preprocessing

In [5]:
def read_image(path):
    if isinstance(path, (bytearray, bytes)):
        path = path.decode("utf-8")

    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (256, 256))
    x = x/255.0
    return x


def read_mask(path):
    if isinstance(path, (bytearray, bytes)):
        path = path.decode("utf-8")
    
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (256, 256))
    x = x/255.0
    x = np.expand_dims(x, axis=-1)
    return x


## Load data using TensorFlow Dataset API

In [6]:
def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float64, tf.float64])
    x.set_shape([256, 256, 3])
    y.set_shape([256, 256, 1])
    return x, y


def tf_dataset(x, y, batch_size=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.shuffle(buffer_size=100)
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch_size)
    dataset = dataset.repeat()
    return dataset


In [7]:
# Dataset
path = "I:\\Database\\CVC-612"
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(path)

# Hyperparameters
batch = 8
lr = 1e-4
epochs = 20

train_dataset = tf_dataset(train_x, train_y, batch_size=batch)
valid_dataset = tf_dataset(valid_x, valid_y, batch_size=batch)

model = build_model()


In [8]:
def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)


opt = tf.keras.optimizers.Adam(lr)
metrics = ["acc", tf.keras.metrics.Recall(), tf.keras.metrics.Precision(), iou]
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=metrics)


In [9]:
callbacks = [
    # ModelCheckpoint("files/model.h5"),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4),
    CSVLogger("files/data.csv"),
    TensorBoard(),
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=False)
]

train_steps = len(train_x)//batch
valid_steps = len(valid_x)//batch

if len(train_x) % batch != 0:
    train_steps += 1
if len(valid_x) % batch != 0:
    valid_steps += 1


In [10]:
model.fit(train_dataset,
          validation_data=valid_dataset,
          epochs=epochs,
          steps_per_epoch=train_steps,
          validation_steps=valid_steps,
          callbacks=callbacks)


Epoch 1/20
2/2 [==============================] - 7s 3s/step - loss: 0.7890 - acc: 0.4964 - recall: 0.4960 - precision: 0.2086 - iou: 0.1511 - val_loss: 0.6957 - val_acc: 0.4502 - val_recall: 0.9239 - val_precision: 0.2529 - val_iou: 0.1592
Epoch 2/20
2/2 [==============================] - 4s 2s/step - loss: 0.7462 - acc: 0.5310 - recall: 0.5871 - precision: 0.2453 - iou: 0.1632 - val_loss: 0.6938 - val_acc: 0.5312 - val_recall: 0.7733 - val_precision: 0.2629 - val_iou: 0.1589
Epoch 3/20
2/2 [==============================] - 4s 2s/step - loss: 0.7083 - acc: 0.5655 - recall: 0.6415 - precision: 0.2743 - iou: 0.1735 - val_loss: 0.6923 - val_acc: 0.5671 - val_recall: 0.7269 - val_precision: 0.2725 - val_iou: 0.1589
Epoch 4/20
2/2 [==============================] - 3s 2s/step - loss: 0.6711 - acc: 0.6065 - recall: 0.6807 - precision: 0.3062 - iou: 0.1837 - val_loss: 0.6910 - val_acc: 0.6204 - val_recall: 0.7079 - val_precision: 0.3003 - val_iou: 0.1590
Epoch 5/20
2/2 [====================